In [ ]:
import holodeck as holo
from holodeck import single_sources, utils, plot, detstats
from holodeck.constants import YR

import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py



# Anisotropy Functions

In [ ]:
NSIDE = 24
LMAX = 8

def healpix_map(hc_ss, hc_bg, nside=NSIDE):
    """ Build mollview array of strains for a healpix map
    
    Parameters
    ----------
    hc_ss : (F,R,L) NDarray
        Characteristic strain of single sources.
    hc_bg : (F,R) NDarray
        Characteristic strain of the background.
    nside : integer
        number of sides for healpix map.

    Returns
    -------
    moll_hc : (NPIX,) 1Darray
        Array of strain at every pixel for a mollview healpix map.
    
    NOTE: Could speed up the for-loops, but it's ok for now.
    """

    npix = hp.nside2npix(nside)
    nfreqs = len(hc_ss)
    nreals = len(hc_ss[0])
    nloudest = len(hc_ss[0,0])

    # spread background evenly across pixels in moll_hc
    moll_hc = np.ones((nfreqs,nreals,npix)) * hc_bg[:,:,np.newaxis]/np.sqrt(npix) # (frequency, realization, pixel)

    # choose random pixels to place the single sources
    pix_ss = np.random.randint(0, npix-1, size=nfreqs*nreals*nloudest).reshape(nfreqs, nreals, nloudest)
    for ff in range(nfreqs):
        for rr in range(nreals):
            for ll in range(nloudest):
                moll_hc[ff,rr,pix_ss[ff,rr,ll]] = np.sqrt(moll_hc[ff,rr,pix_ss[ff,rr,ll]]**2
                                                          + hc_ss[ff,rr,ll]**2)
                
    return moll_hc

def sph_harm_from_map(moll_hc, lmax=LMAX):
    """ Calculate spherical harmonics from strains at every pixel of 
    a healpix mollview map.
    
    Parameters
    ----------
    moll_hc : (F,R,NPIX,) 1Darray
        Characteristic strain of each pixel of a healpix map.
    lmax : int
        Highest harmonic to calculate.

    Returns
    -------
    Cl : (F,R,lmax+1) NDarray
        Spherical harmonic coefficients 
        
    """
    nfreqs = len(moll_hc)
    nreals = len(moll_hc[0])

    Cl = np.zeros((nfreqs, nreals, lmax+1))
    for ff in range(nfreqs):
        for rr in range(nreals):
            Cl[ff,rr,:] = hp.anafast(moll_hc[ff,rr], lmax=lmax)

    return Cl

def sph_harm_from_hc(hc_ss, hc_bg, nside = NSIDE, lmax = LMAX):
    """ Calculate spherical harmonics and strain at every pixel
    of a healpix mollview map from single source and background char strains.

    Parameters
    ----------
    hc_ss : (F,R,L) NDarray
        Characteristic strain of single sources.
    hc_bg : (F,R) NDarray
        Characteristic strain of the background.
    nside : integer
        number of sides for healpix map.

    Returns
    -------
    moll_hc : (F,R,NPIX,) 2Darray
        Array of strain at every pixel for a mollview healpix map.
    Cl : (F,R,lmax+1) NDarray
        Spherical harmonic coefficients 
    
    """
    moll_hc = healpix_map(hc_ss, hc_bg, nside)
    Cl = sph_harm_from_map(moll_hc, lmax)

    return moll_hc, Cl



##########################################################################################
################################### PLOTTING FUNCTIONS ###################################
##########################################################################################

def plot_Cl(Cl, lvals, fobs_nHz):
    """ Plot C_l vs l for all frequencies of a single realization.
    
    """
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')
    colors=cm.rainbow(np.linspace(0,1,len(Cl)))

    for ff in range(len(Cl)):
        if ff in (0, 10, 20, 30, 39):
            label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
        else: label = None
        ax.plot(lvals[:], Cl[ff,:], c=colors[ff], alpha=0.5, label=label,
                marker='o')
        
    ax.legend()
    return fig

def plot_ClC0(Cl, lvals, fobs_nHz):
    """ Plot C_l/C_0 for all frequencies of a single realization.
    
    """
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')
    colors=cm.rainbow(np.linspace(0,1,len(Cl)))

    for ff in range(len(Cl)):
        if ff in (0, 10, 20, 30, 39):
            label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
        else: label = None
        ax.plot(lvals[:], Cl[ff,:]/Cl[ff,0,np.newaxis], c=colors[ff], alpha=0.5, label=label,
                marker='o')
        
    ax.legend()
    return fig

def plot_llp1Cl(Cl, lvals, fobs_nHz):
    """ Plot l(l+1)C_l vs l for all frequencies of a single realization.
    
    """
    fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$\ell(\ell+1)C_\ell$', xscale='linear')
    colors=cm.rainbow(np.linspace(0,1,len(Cl)))

    xx = lvals[1:]
    yy = (lvals*(lvals+1))[np.newaxis,1:]*Cl[:,1:]
    for ff in range(len(Cl)):
        if ff in (0, 10, 20, 30, 39):
            label= ('$f$ = %.2f nHz' % fobs_nHz[ff])
        else: label = None
        ax.plot(xx, yy[ff], c=colors[ff], alpha=0.5, label=label,
                marker='o')
        
    ax.legend()
    return fig

def plot_Clg0(Cl, fobs):
    """ Plot C_l>0/C_0 vs f for many realizations.
    
    """

    fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$', xscale='linear')
    xx = fobs*YR
    print(xx.shape)
    print(np.sum(Cl[:,:,1:], axis=2).shape)
    print(Cl[:,:,0,np.newaxis].shape)
    yy = np.sum(Cl[:,:,1:], axis=2)/Cl[:,:,0]
    print(yy.shape)
    for rr in range(len(Cl[0])):
        ax.plot(xx, yy[:,rr], alpha=0.5, marker='o')

    ax.plot(xx,np.median(yy, axis=1), color='k')
        
    ax.legend()
    return fig

def map_from_sph_harm(Cl, nside=NSIDE, lmax=LMAX):
    nfreqs = len(Cl)
    nreals = len(Cl[0])
    npix = hp.nside2npix(nside)
    moll_sh = np.zeros((nfreqs,nreals,npix))
    for ff in range(nfreqs):
        for rr in range(nreals):
            moll_sh[ff,rr,:] = hp.synfast(Cl[ff,rr], nside, lmax=lmax)
    return moll_sh

# Ranking Sample Functions

In [ ]:
def amp_to_hc(amp_ref, fobs, dfobs):
    hc = amp_ref*np.sqrt(fobs/dfobs)
    return hc

def rank_samples(hc_ss, hc_bg, fobs, dfobs, amp_ref):
    """ Sort samples by those with f=1/yr char strains closest to some reference value.
    
    Parameters
    ----------
    hc_ss : (N,F,R,L) NDarray
        Characteristic strain of the loudest single sources.
    hc_bg : (N,F,R) NDarray
        Characteristic strain of the background.
    fobs : (F,)
        Observed GW frequency
    dfobs : (F,)
        Observed GW frequency bin widths.
    amp_ref : scalar
        Reference strain amplitude at f=1/yr

    Returns
    -------
    nsort : (N,) 1Darray
        Indices of the param space samples sorted by proximity to the reference 1yr amplitude.
    """
    # find bin with 1/yr
    fidx = (np.abs(fobs - 1/YR)).argmin()

    # find reference (e.g. 12.5 yr) char strain
    hc_ref = amp_to_hc(amp_ref, fobs[fidx], dfobs[fidx])

    # select 1/yr median strains of samples
    hc_1yr = np.sqrt(hc_bg[:,fidx,:]**2 + np.sum(hc_ss[:,fidx,:,:]**2, axis=-1)) # (N,R)
    hc_1yr = np.median(hc_1yr, axis=1) 

    # sort by closest
    nsort = np.argsort(np.abs(hc_1yr-hc_ref))

    return nsort

# Read in Library

ss16 has 10 loudest, definitely will replace this with a better library

In [ ]:
sspath = '/Users/emigardiner/GWs/holodeck/output/2023-05-12-mbp-ss16_n10_r10_f70_d12.5_l10_p0/'
hdfname = sspath+'ss_lib.hdf5'
ssfile = h5py.File(hdfname, 'r')
print(list(ssfile.keys()))
hc_ss = ssfile['hc_ss'][...]
hc_bg = ssfile['hc_bg'][...]
fobs = ssfile['fobs'][:]
dfobs = ssfile['dfobs'][:]
ssfile.close()

shape = hc_ss.shape
nsamps, nfreqs, nreals, nloudest = shape[0], shape[1], shape[2], shape[3]
print('N,F,R,L =', nsamps, nfreqs, nreals, nloudest)

In [ ]:
nsort = rank_samples(hc_ss, hc_bg, fobs, dfobs, amp_ref=2.4*10**-15)

### Calculate pixel strains and harmonic coefficients

In [ ]:
nside = NSIDE
npix = hp.nside2npix(nside)
lmax = LMAX
nbest = 10
Cl_best = np.zeros((nbest, nfreqs, nreals, lmax+1 ))
moll_hc_best = np.zeros((nbest, nfreqs, nreals, npix))
for nn in range(nbest):
    print('on nn=%d out of nbest=%d' % (nn,nbest))
    moll_hc_best[nn,...], Cl_best[nn,...] = sph_harm_from_hc(hc_ss[nsort[nn]], 
                                            hc_bg[nsort[nn]],
                                            nside=nside, lmax=lmax)

### Plot Results

In [ ]:
print(nfreqs)

In [ ]:
lvals = np.arange(lmax+1)
fobs_nHz = fobs*10**9
fobs_yrs = fobs*YR

In [ ]:
fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell$', xscale='linear')

xx = lvals # (l)
yy = Cl_best # (B,F,R,l)

colors = cm.rainbow(np.linspace(1, 0, 31))
rr=0
for ff in (0,5,10,15,20, 25, 30):
    label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
    ax.plot(xx, np.median(yy[:,ff,rr,:], axis=0), color=colors[ff], label=label)
    for pp in [50, 98]:
        percs = pp/2
        percs = [50-percs, 50+percs]
        ax.fill_between(xx, *np.percentile(yy[:,ff,rr,:], percs, axis=0), alpha=0.25, color=colors[ff])
plt.legend()
ax.set_title('50%% and 98%% confidence intervals from 0th realizations of the %d best samples' % nbest)

In [ ]:
fig, ax = plot.figax(figsize=(8,5), xlabel='$\ell$', ylabel='$C_\ell/C_0$', xscale='linear')

xx = lvals[1:] # (l)
yy = Cl_best[:,:,:,1:]/Cl_best[:,:,:,0,np.newaxis] # (B,F,R,l)

colors = cm.rainbow(np.linspace(1, 0, 31))
rr=0
for ff in (0,5,10,15,20, 25, 30):
    label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
    ax.plot(xx, np.median(yy[:,ff,rr,:], axis=0), color=colors[ff], label=label)
    for pp in [50, 98]:
        percs = pp/2
        percs = [50-percs, 50+percs]
        ax.fill_between(xx, *np.percentile(yy[:,ff,rr,:], percs, axis=0), alpha=0.25, color=colors[ff])
ax.legend()
ax.set_title('50%% and 98%% confidence intervals from 0th realizations of the %d best samples' % nbest)

In [ ]:
fig, ax = plot.figax(figsize=(8,5), xlabel=plot.LABEL_GW_FREQUENCY_YR, ylabel='$C_{\ell>0}/C_0$')

xx = fobs_yrs
yy = np.sum(Cl_best[:,:,:,1:], axis=-1)/Cl_best[:,:,:,0,] # (B,F,R)

colors = cm.rainbow(np.linspace(1, 0, nbest))
rr=0
for bb in range(nbest):
    # label= ('$f$ = %.2f nHz = %.2f /yr' % (fobs_nHz[ff], fobs_yrs[ff]))
    label='p = %d' % nsort[bb]
    ax.plot(xx, np.median(yy[bb,:,:], axis=-1), color=colors[bb], label=label)
    for pp in [50, 98]:
        percs = pp/2
        percs = [50-percs, 50+percs]
        ax.fill_between(xx, *np.percentile(yy[bb,:,:], percs, axis=-1), alpha=0.25, color=colors[bb])
ax.legend()
ax.set_title('50%% and 98%% confidence intervals from %d realizations of the %d best samples' % (nreals,nbest))